In [1]:
%matplotlib widget
#%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point, LineString, MultiPoint, Polygon
from shapely.wkt import loads
from scipy.io import loadmat
from scipy import spatial
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc
import os
import glob
from scipy.spatial.ckdtree import cKDTree
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils

In [3]:
def nearest_neighbours(points, coords, points_required = 1, max_distance = 250.):
    """

    :param points: array of points to find the nearest neighbour for
    :param coords: coordinates of points
    :param points_required: number of points to return
    :param max_distance: maximum search radius
    :return:
    """
    # Initialise tree instance
    kdtree = cKDTree(data=coords)

    # iterate throught the points and find the nearest neighbour
    distances, indices = kdtree.query(points, k=points_required,
                                      distance_upper_bound=max_distance)
    print(distances)
    # Mask out infitnite distances in indices to avoid confusion
    mask = ~np.isfinite(distances)

    distances[mask] = np.nan

    return distances, indices

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=100.)
    if np.isnan(d[0]):
        return None

    else:
        near_ind = i[0]
    


        return grid_dict[var][near_ind]


In [4]:
# bring in the inversion ready data

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\garjmcmctdem_4\combined\lines\rjmcmc"

gdf = aseg_gdf2.read(infile).df()

In [5]:
gdf['fiducial ']

0       1124778.5
1       1126026.5
2       1126447.5
3       1127583.5
4       1127983.0
          ...    
1452    1983849.0
1453    1983899.0
1454    1983949.0
1455    1983999.0
1456    1984049.0
Name: fiducial , Length: 1457, dtype: float64

In [6]:
# Now add the path to the netcdf file based on the naming convention

root = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\garjmcmctdem_4\combined\pmaps"
#root = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\garjmcmctdem_3\run7\output\pmaps"

gdf['pmap_path']= ''

for index, row in gdf.iterrows():
    fn= ['seq', str(row['uniqueid ']).zfill(8), str(row['line ']), "{:.6f}".format(row['fiducial ']), 'nc' ]
    gdf.at[index, 'pmap_path'] = os.path.join(root, '.'.join(fn))

In [7]:
# This column is for interpreting the Oolloo-Jinduckin interface

gdf['topJind_interp'] = np.nan

In [8]:
lines = gdf['line '].unique()

In [9]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\DalyR_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


In [10]:
# Since we will be wanting to plot the sections lets first grid them

# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 16., 4.


In [11]:

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\lci\hdf5"

if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)

gridded_vars ={}

# Saved these out to save time

#gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines,
#                                    resampling_method = 'linear', save_hdf5 = True,
#                                   return_dict = True, hdf5_dir = hdf5_dir)

for line in lines:
    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    f = h5py.File(infile, 'r')
    gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    f = None
    gc.collect()

In [12]:
gridded_vars.keys()

dict_keys([102901, 103001, 103101, 103201, 103301, 103401, 103501, 103601, 103701, 103801, 103901, 104001, 104101, 110102, 103502, 104501, 104102, 104201, 104301, 104401, 104601, 104701, 104801, 104901, 105001, 105101, 105201, 105301, 105401, 105501, 105701, 105801, 105901, 106001, 106101, 106201, 110103, 105902, 106401, 106501, 106601, 106701, 106801, 106901, 107001, 107101, 107201, 107301, 107401, 108501, 108601, 108701, 108801, 109001, 109201, 110201, 109401, 109502, 107501, 107601, 107701, 107801, 107901, 108001, 108101, 108201, 108301, 108401, 110202, 108901, 109301, 109402, 110001, 110002, 109202, 109302, 109504, 109303, 109101])

In [13]:
# Now we bring in the Oolloo Jinduckin contact to plot

#inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\topOolloo_BOM.tif"
inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\tJind_BOM.tif"

contact_dataset = rasterio.open(inRaster)

contact_elev = contact_dataset.read(1)

contact_elev[contact_elev == contact_dataset.get_nodatavals()] = np.nan


In [14]:
# We want to find the top of the Jinduckin for every gridded lci section

for line in gridded_vars.keys():
    
    # Get the coordinates
    utm_coords = utm_coords = np.column_stack((gridded_vars[line]['easting'],
                                               gridded_vars[line]['northing']))
    # sample the raster at the coordinates
    
    elevGen = contact_dataset.sample(utm_coords)
    
    gridded_vars[line]['topJind_elev'] = np.nan*np.ones(shape = utm_coords.shape[0],
                                                 dtype = np.float)
    
    for i in range(utm_coords.shape[0]):
        
        jindElev =  next(elevGen)

        if not np.isclose(jindElev,contact_dataset.get_nodatavals()[0]):
            gridded_vars[line]['topJind_elev'][i] = jindElev


In [15]:
# generator for sampling the bom raster
contElev = contact_dataset.sample(gdf[['easting ', 'northing ']].values)

# Add the distance along the gridded lines to the dataframe


gdf['point_ind_lci'] = -9999

gdf['topJind_depth'] = np.nan

for index, row in gdf.iterrows():

    # get the line and coordinates
    line = row['line ']
    
    xy = np.array([row[['easting ','northing ']].values])
    
    gdf.loc[index,'dist_along_line'] = xy_2_var(gridded_vars[line],
                                               xy,'grid_distances')
    
    gdf.loc[index,'point_ind_lci'] = kdtree.query(row[['easting ', 'northing ']].values)[1]
    
    jindElev = next(contElev)[0]
    
    if not np.isclose(jindElev,contact_dataset.get_nodatavals()[0]):
        
        gdf.loc[index,'topJind_depth'] = row['elevation '] - jindElev

In [16]:
# Now we bring in the AEM grids

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)



In [17]:
gdf['topJind_interp']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1452   NaN
1453   NaN
1454   NaN
1455   NaN
1456   NaN
Name: topJind_interp, Length: 1457, dtype: float64

In [39]:
# This function stores the top of the conductor in the dataframe on a click
def onclick(event):
    if event.xdata != None and event.ydata != None:
        gdf.at[ind, 'topJind_interp'] = event.ydata


def extract_data():
    """
    FUnction for extracting all the AEM data from the netCDF files
    """
    rj_dat = netCDF4.Dataset(row['pmap_path'])

    freq = rj_dat['log10conductivity_histogram'][:].data.astype(np.float)

    cond_pdf = freq / freq.sum(axis =1)[0]

    cond_pdf[cond_pdf == 0] = np.nan
    
    cp_freq = rj_dat["interface_depth_histogram"][:].data.astype(np.float)
    
    cp_pdf = cp_freq / freq.sum(axis =1)[0]
    
    laybins = rj_dat['nlayers_histogram'][:].data
    
    lay_prob = laybins / freq.sum(axis =1)[0]
    
    condmin, condmax = rj_dat.vmin, rj_dat.vmax 
    
    ncond_cells = rj_dat.dimensions['value'].size
    
    cond_cells = np.linspace(condmin, condmax, ncond_cells)
    
    pmin, pmax = rj_dat.pmin, rj_dat.pmax
    
    
    depth_cells = rj_dat['depth'][:].data
    
    extent = [cond_cells.min(), cond_cells.max(), depth_cells.max(), depth_cells.min()]
    
    mean = np.power(10,rj_dat['mean_model'][:].data)
    p10 = np.power(10,rj_dat['p10_model'][:].data)
    p50 = np.power(10,rj_dat['p50_model'][:].data)
    p90 = np.power(10,rj_dat['p90_model'][:].data)
    
    lci_cond = lci_dat['conductivity'][point_ind_lci].data
    lci_depth_top = lci_dat['layer_top_depth'][point_ind_lci].data
    
    lci_doi = lci_dat['depth_of_investigation'][point_ind_lci].data
    
    misfit = np.sqrt(rj_dat['misfit'][:].data/rj_dat.ndata)
    
    burnin = rj_dat.nburnin
    nsamples = rj_dat.nsamples
    sample_no = rj_dat['convergence_sample'][:].data
    nchains = rj_dat.nchains
    
    line = int(rj_dat.line)
    
    return {'conductivity_pdf': cond_pdf, "change_point_pdf": cp_pdf, "conductivity_extent": extent,
           'cond_p10': p10, 'cond_p50': p50, 'cond_p90': p90, 'cond_mean': mean, 'depth_cells': depth_cells,
           'nlayer_bins': laybins, 'nlayer_prob': lay_prob, 'nsamples': nsamples, 'ndata': rj_dat.ndata,
           "nchains": nchains, 'burnin': burnin, 'misfit': misfit, 'sample_no': sample_no, 'cond_cells': cond_cells, 'lci_cond': lci_cond,
           'lci_depth_top': lci_depth_top, 'lci_doi': lci_doi, 'line': line}
    
def DR_plot(D, outfile = None):
    fig = plt.figure(figsize = (12,10))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '5in'

    ax1 = fig.add_axes([0.05, 0.35, 0.35, 0.62])
    ax2 = fig.add_axes([0.45, 0.35, 0.2, 0.62])
    ax3 = fig.add_axes([0.70, 0.52, 0.2, 0.2])
    ax4 = fig.add_axes([0.72, 0.32, 0.16, 0.16])
    ax5 = fig.add_axes([0.1, 0.18, 0.76, 0.05])
    ax6 = fig.add_axes([0.1, 0.05, 0.76, 0.13])
    ax7 = fig.add_axes([0.70, 0.78, 0.2, 0.2])
    cbar_ax1 = fig.add_axes([0.05, 0.29, 0.35, 0.01])
    cbar_ax2 = fig.add_axes([0.88, 0.05, 0.01, 0.2])
    
    panel_kwargs = [{'title': '',
                      'color': 'black',
                      'ylabel': 'data \n residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 450.,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': 2.,
                      'cmap': 'jet',
                      'ylabel': 'elevation \n (mAHD)',
                      'vertical_exaggeration': 5.0}]


    # Plot probability map
    
    im = ax1.imshow(D['conductivity_pdf'], extent = D['conductivity_extent'],
                    aspect = 'auto', cmap = 'rainbow')
    
    #  PLot the median, and percentile plots
    ax1.plot(np.log10(D['cond_p10']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p10')
    ax1.plot(np.log10(D['cond_p90']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p90')
    ax1.plot(np.log10(D['cond_p50']), D['depth_cells'], c = 'k',label = 'p50')
    ax1.plot(np.log10(D['cond_mean']), D['depth_cells'], c = 'grey',label = 'mean')
    
    ax1.set_xticklabels([round(10 ** float(x), 4) for x in ax1.get_xticks()])

    # for lci layered model we do some processing
    lci_expanded = np.zeros(shape=2 * len(D['lci_cond']) + 1,
                                 dtype=np.float)

    lci_expanded[1:] = np.repeat(D['lci_cond'], 2)

    depth_expanded = (np.max(D['lci_depth_top']) + 10) * np.ones(shape=len(lci_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(D['lci_depth_top'], 2)

    ax1.plot(np.log10(lci_expanded), depth_expanded, c = 'pink',
             linestyle = 'dashed', label = 'lci')
    ax1.plot(ax1.get_xlim(), [D['lci_doi'], D['lci_doi']], c = 'yellow',
             label = 'LCI doi')
    ax1.set_title('rj-MCMC probability map- FID: ' + str(row['fiducial ']))
    ax1.set_ylabel('depth (mBGL)')
    ax1.set_xlabel('Conductivity (S/m)')
    ax1.grid(which = 'both')
    ax1.set_xlim(D['conductivity_extent'][0], D['conductivity_extent'][1] )
    
    ax1.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#100.,0)#
    
    
    ax1.plot(ax1.get_xlim(), [row['topJind_depth'], row['topJind_depth']],
            c = 'green', linewidth = 2, label = 'top Jinduckin - BOM')
    ax1.legend()
    ax2.plot(D['change_point_pdf'], D['depth_cells'], label = 'P(change point)')
    ax2.set_ylim(ax2.get_ylim()[::-1])
    ax2.set_yticks(np.arange(0, 500, 20.))
    ax2.set_title('change point probability')
    ax2.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#(100.,0)
    ax2.plot(ax2.get_xlim(), [row['topJind_depth'], row['topJind_depth']],
            c = 'green', linewidth = 2, label = 'top Jinduckin - BOM')
    ax2.legend()
    ax2.grid(which = 'both')
    
    
    

    ax3.imshow(contact_elev, 
               extent = [contact_dataset.bounds[0], contact_dataset.bounds[2],
                         contact_dataset.bounds[1], contact_dataset.bounds[3]],
              cmap= 'viridis')#, vmin = -2., vmax = 2.)
    #ax3.set_xlim(D['easting'] - 10000., D['easting'] + 10000.)
    #ax3.set_ylim(D['northing'] - 10000., D['northing'] + 10000.)
    ax3.plot(D['easting'],D['northing'], 'x', c = 'r')
    ax3.set_title('Oolloo-Jinduckin contact', fontsize=10)
    #ax3.set_title("Top Oolloo")
    ax3.tick_params(axis='both', which='major', labelsize=8)
    ax3.tick_params(axis='both', which='minor', labelsize=8)
    # Ax3 will be our location
    sample = D['sample_no'][:]
    
    # Add the misfit
    for i in range(D['misfit'].shape[0]):
       
        misfits = D['misfit'][i]
        ax4.plot(sample, misfits)

    ax4.plot([1, D['nsamples']], [1,1], 'k')
    ax4.plot([D['burnin'], D['burnin']],[0.1,1e4], 'k')
    ax4.set_xlim([1, D['nsamples']])
    ax4.set_ylim(0.01, 1e4)

    ax4.set_xscale('log')
    ax4.set_yscale('log')

    ax4.set_xlabel("sample #")
    ax4.set_ylabel("Normalised misfit")
    
    # conductivity plot
    
    ax7.imshow(np.log10(cond[9]), extent = [cond_dataset.bounds[0],
                                  cond_dataset.bounds[2],
                                  cond_dataset.bounds[1], 
                                  cond_dataset.bounds[3]],
              cmap = 'jet',
              vmin = np.log10(panel_kwargs[1]['vmin']*1000.),
              vmax = np.log10(panel_kwargs[1]['vmax']*1000.))
    
    ax7.set_xlim(D['easting'] - 10000., D['easting'] + 10000.)
    ax7.set_ylim(D['northing'] - 10000., D['northing'] + 10000.)
    ax7.plot(D['easting'],D['northing'],  'x', c = 'k')
    
    p1 = [gridded_vars[line]['easting'][0], gridded_vars[line]['easting'][-1]]
    p2 = [gridded_vars[line]['northing'][0], gridded_vars[line]['northing'][-1]]
    ax7.plot(p1, p2, 'k', linewidth = 0.5)
    ax7.set_title('LCI depth slice 61.8-71.6 mBGL', fontsize=10)
    ax7.tick_params(axis='both', which='major', labelsize=8)
    ax7.tick_params(axis='both', which='minor', labelsize=8)
    cb1 = fig.colorbar(im, cax=cbar_ax1, orientation='horizontal')
    cb1.set_label('probabilitiy', fontsize=10)

    res1 = plot_utils.plot_single_line(ax5, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax5.set_title('LCI conductivity section - ' + str(line))

    im2 = plot_utils.plot_grid(ax6, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])
    
    # PLot jinduckin elevation onto section
    ax6.plot(gridded_vars[line]['grid_distances'],
            gridded_vars[line]['topJind_elev'], c = 'k',
            linestyle = 'dashed')
    
    # Add colorbars

    ax6.plot([dist, dist], [-500, 500], 'pink')
    
        
    cb2 = fig.colorbar(im2, cax=cbar_ax2, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)
    
    ax5.set_xlim(dist, 10000.,
                 dist + 10000.)
    ax6.set_xlim(dist - 10000., 
                 dist + 10000.)

    
    
    return fig

def gen(df):
    for index, row in df.iterrows():
        yield index, row

In [40]:
lines

array([102901, 103001, 103101, 103201, 103301, 103401, 103501, 103601,
       103701, 103801, 103901, 104001, 104101, 110102, 103502, 104501,
       104102, 104201, 104301, 104401, 104601, 104701, 104801, 104901,
       105001, 105101, 105201, 105301, 105401, 105501, 105701, 105801,
       105901, 106001, 106101, 106201, 110103, 105902, 106401, 106501,
       106601, 106701, 106801, 106901, 107001, 107101, 107201, 107301,
       107401, 108501, 108601, 108701, 108801, 109001, 109201, 110201,
       109401, 109502, 107501, 107601, 107701, 107801, 107901, 108001,
       108101, 108201, 108301, 108401, 110202, 108901, 109301, 109402,
       110001, 110002, 109202, 109302, 109504, 109303, 109101],
      dtype=int64)

In [65]:
# Create a generator to iterate through the pandas dataframe

line = 108701

n = 3

df_line = gdf[gdf['line '] == line]

print(len(df_line))


print(len(df_line.iloc[::n,:]))

cond_gen = gen(df_line.iloc[::n,:])
#cond_gen = gen(df_line.iloc[-1:,:])


11
4


In [69]:
# Define some coordinates to investigate


ind, row = next(cond_gen)

point_ind_lci = row['point_ind_lci']

# Get the nearest lci point

easting, northing = row['easting '], row['northing ']

plt.close('all')

D = extract_data()

D['easting'], D['northing'] = easting, northing

line = row['line ']

# Find distance along the lci section
dist = xy_2_var(gridded_vars[line],
                 np.array([[easting, northing]]),
                         'grid_distances')

print(easting)
print(northing)

fig = DR_plot(D)
#plt.savefig(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\rjplots\\Florina_" + row['BORE_NO'] + ".png", dpi = 150)
cid=  fig.canvas.mpl_connect('button_press_event', onclick)

cols = ['uniqueid ', 'survey ', 'date ', 'flight ', 'line ', 'fiducial ',
       'easting ', 'northing ', 'elevation ', 'nchains ', 'nsamples ',
       'nburnin ', 'sampletime ', 'misfit_lowest ', 'misfit_average ',
        'topJind_interp']

mask = gdf["topJind_interp"].notnull()

gdf[cols].to_csv(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\Jinduckin_Oolloo_rj_interp_2.csv")

plt.show()


767844.2
8400589.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:123: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:185: RuntimeWarning: invalid value encountered in log10


In [70]:
mask = gdf["topJind_interp"].notnull()

gdf[mask][cols].to_csv(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\Jinduckin_Oolloo_rj_interp_3.csv")


In [47]:
np.nanmax(gdf['topJind_interp'].values)

270.69876448915824

In [22]:
np.sqrt(0.01)

0.1

In [48]:
plt.savefig(r"C:\temp\slackplot.png", dpi= 300)